In [ ]:
import sys
sys.path.append('../')
import pipeline.sql as plsql

import pandas as pd
import numpy as np

from plotnine import *

from mizani.breaks import date_breaks
from mizani.formatters import date_format
from datetime import date

theme_set(theme_bw())

In [ ]:
def custom_date_format2(breaks):
    """
    Function to format the date
    """
    res = []
    for x in breaks:
        # First day of the year
        if x.month == 1 and x.day == 1:
            fmt = '%Y'
        # Every other month
        elif x.month % 2 != 0:
            fmt = '%b'
        else:
            fmt = ''

        res.append(date.strftime(x, fmt))

    return res

def custom_date_format3(breaks):
    """
    Function to format the date
    """
    res = []
    for x in breaks:
        # First day of the year
        if x.month == 1:
            fmt = '%Y'
        else:
            fmt = '%b'

        res.append(date.strftime(x, fmt))

    return res


def custom_date_breaks(width=None):
    """
    Create a function that calculates date breaks

    It delegates the work to `date_breaks`
    """
    def filter_func(limits):
        breaks = date_breaks(width)(limits)
        # filter
        return [x for x in breaks if x.month % 2]

    return filter_func

In [ ]:
engine = plsql.create_engine('../config.yaml')

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select *
    from raw.inspections_complete
    limit 3;"""
df = pd.read_sql_query(qry, engine)
df.columns

In [ ]:
df.head()

In [ ]:
df.derechofund.head()

Companies inspected by year

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select agno, count( distinct rutempresamask ) as count_insp
    from raw.inspections_complete
    group by agno
    order by agno desc;"""
tt1 = pd.read_sql_query(qry, engine)

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select agno, count( distinct rutmask ) as count_tax
    from raw.taxes
    group by agno
    order by agno desc;"""
tt2 = pd.read_sql_query(qry, engine)

In [ ]:
tab_merge = pd.merge(tt1, tt2, on='agno')
tab_merge['ratio'] = tab_merge['count_insp']/tab_merge['count_tax']
(ggplot(tab_merge[tab_merge['agno']< 2016], 
        aes('agno', 'ratio')) + 
    geom_line() +
    geom_point(color = "#e91d63", size = 2) +
    ylim(0, .3) + 
    ylab('Inspected / Number') + 
    xlab('Year')+ 
    scale_x_continuous(breaks = (2006, 2008, 2010, 2012, 2014, 2016)) +
    theme(figure_size = (5, 3)))

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select agno, count(*) as count, sum(infra) as inspections
    from raw.inspections_complete
    group by agno
    order by inspections desc;"""
pd.read_sql_query(qry, engine)

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select grae, crae, count(*) as count, sum(infra) as infractions
    from raw.inspections_complete
    group by grae, crae
    order by infractions desc;"""
pd.read_sql_query(qry, engine)

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, avg(emptrabhombres) as workers, count(*) as inspections, sum(infra) as infractions
    from raw.inspections_complete
    group by rutempresamask;"""
tab_all = pd.read_sql_query(qry, engine)

In [ ]:
(ggplot(tab_all, aes('inspections', 'infractions')) + 
    geom_point(alpha = .5)+ 
    theme(figure_size = (5, 4)))

In [ ]:
(ggplot(tab_all, aes('inspections', 'workers')) + 
    geom_point(alpha = .5) + 
    xlab('Average number of workers') + 
    theme(figure_size = (5, 4)))

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select crae, grae, count( distinct rutempresamask)
    from raw.inspections_complete
    group by crae, grae
    order by count;"""
tab_numemp = pd.read_sql_query(qry, engine)

In [ ]:
tab_numemp

In [ ]:
tab_numemp['grae'] = tab_numemp['grae'].astype(str).str[0:33].str.lower()
tab_numemp['mining'] = tab_numemp['crae'] == '103'
tab_numemp['agriculture'] = tab_numemp['crae'] == '101'
tab_numemp['grae'] = pd.Categorical(tab_numemp['grae'], categories=tab_numemp.grae.tolist())

In [ ]:
(ggplot(tab_numemp, aes(x = 'grae', y = 'count', 
                       fill = 'agriculture')) + 
 geom_bar(stat = 'identity') + 
 xlab("Economic Activity") + 
 ylab('number of inspections') +
 scale_fill_manual(values = ("gray", "#e91d63"),
                   guide = False) + 
 coord_flip() + 
 theme(figure_size = (4, 5)) )

## Mining and quarrying (code 103)	

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select *
    from raw.inspections_complete
    where crae = '103';"""
df_mining = pd.read_sql_query(qry, engine)
#df_mining.head()
#df_mining.columns

Mining has 2725 companies

In [ ]:
len(df_mining['rutempresamask'].unique())

In [ ]:
df_mining['gcae_rec'] = df_mining['gcae'].astype(str).str[0:35]
df_mining['gcae_rec'] = pd.Categorical(df_mining['gcae_rec'], categories=df_mining.gcae_rec.value_counts().index.tolist())

In [ ]:
(ggplot(df_mining, aes(x = 'gcae_rec')) + 
 geom_histogram(stat = 'count', fill = 'purple') + 
 xlab("Economic Activity") + 
 ylab('number of inspections') + 
 theme(axis_text_x = element_text(rotation = 90, vjust = 1, hjust = 1),
       figure_size = (5, 3)))

In [ ]:
def my_agg(x):
    names = {
        'max_date': x['datereg'].max(),
        'min_date': x['datereg'].min(),
        'num_days': max(x['datereg']) - min(x['datereg']),
        'inspections': x['idfiscalizacion'].count()}
    return pd.Series(names, index=['max_date', 'min_date', 'num_days', 'inspections'])
tab_agg = df_mining.groupby('rutempresamask').apply(my_agg).sort_values('num_days', ascending = False).reset_index() 
tab_agg.head()
tab_agg.dtypes

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select gcae, sum(infra) as num_infra, count (*) as num_insp
    from raw.inspections_complete
    where crae = '103' and infra > 0
    group by gcae
    order by num_infra desc;"""
tab = pd.read_sql_query(qry, engine)

In [ ]:
tab['gcae_rec'] = tab['gcae'].astype(str).str[0:35]
tab['ratio'] = tab['num_infra']/tab['num_insp']
tab['ratio_lab'] = round(tab['num_infra']/tab['num_insp'], 0)

In [ ]:
(ggplot(tab, aes(x = 'gcae_rec')) + 
 #geom_bar(stat = 'identity', fill = 'gray') + 
 geom_linerange(aes(ymin = 1, ymax = 'ratio'), size = 3) + 
 geom_text(aes(y = 'ratio+.1', label = 'ratio_lab'), size = 8) + 
 geom_hline(yintercept = 1, color = "salmon", size = 1) + 
 ylab('infractions / inspections') + 
 xlab('Mining subcategories') + 
 theme(axis_text_x = element_text(rotation = 90, vjust = 1, hjust = 1),
       figure_size = (6, 4)))

### Copper extraction 

In [ ]:
df_mining.loc[:, ["gcae", "ccae"]]
df_copper = df_mining[df_mining['ccae'] == '133000']
print ('Number of inspections: ' + str(df_copper.shape[0]))
print ('Number of facilities: ' + str(len(df_copper['rutempresamask'].unique()))) 
df_copper.head()

### Correlations

Number of workers, number of inspections and number of infractions for copper extraction facilities. 

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, avg(emptrabhombres) as workers, count(*) as inspections, sum(infra) as infractions
    from raw.inspections_complete
    where ccae = '133000'
    group by rutempresamask;"""
tab = pd.read_sql_query(qry, engine)
tab['ind'] = tab['inspections'] > 200
tab.head()

#### Scatter plots

Data story facility highlighted in red.

In [ ]:
(ggplot(tab, aes('inspections', 'workers')) + 
    geom_point(aes( color = 'ind'), size = 2, alpha = .5) + 
    scale_color_manual(values = ("black", "red"),
                   guide = False) +
    theme(figure_size = (5, 4)))

In [ ]:
(ggplot(tab[pd.isnull(tab['infractions']) == False], aes('inspections', 'infractions')) + 
    geom_point(aes( color = 'ind'), size = 2, 
              alpha = .7) + 
    scale_color_manual(values = ("black", "#e91d63"),
                   guide = False) + 
    geom_abline(intercept = 0, slope = 1, color = "salmon") +
    theme(figure_size = (5, 4)))

### Number of inspections

In [ ]:
tab['inspections'].describe()

In [ ]:
tab[tab['inspections'] > 500]

### Number of infractions

In [ ]:
tab['infractions'].describe()

In [ ]:
tab[tab['infractions'] > 200].rutempresamask

### Number of employees

Number of employees by inspection

In [ ]:
df_mining['emptrabhombres'].describe()

In [ ]:
(ggplot(df_mining[pd.isnull(df_mining['emptrabhombres']) == False], 
        aes(x = 'grae', y = 'emptrabhombres')) + 
    geom_boxplot() + 
    ylab("number of workers") + 
    xlab('') +
#    scale_y_log10() + 
    coord_flip()+ 
    theme(figure_size = (10, 3)))

Average number of employees per company from inspections information. 

In [ ]:
tab['workers'].describe()

In [ ]:
(ggplot(tab[pd.isnull(tab['workers']) == False], 
        aes(x = '1', y = 'workers')) + 
    geom_boxplot() + 
    ylab("average number of employees") + 
    coord_flip() + 
    theme(figure_size = (7, 3)))

In [ ]:
df_mining[df_mining['emptrabhombres'] == 60000]

In [ ]:
tab[tab['workers'] > 10000]

In [ ]:
tab[tab['inspections']> 500]

### Urgent and Human Rights matters inspected

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, sum(cast(urgencia as int)) as urgents, 
    sum(derechofund) as humanrights,
    sum(infra) as infractions, count(*) as inspections
    from raw.inspections_complete
    where ccae = '133000'
    group by rutempresamask;"""
tab_ins = pd.read_sql_query(qry, engine)

In [ ]:
tab_ins.head()

The company that has a large amount of insepctions also has the highest number of urgent matters inspected. 

In [ ]:
(ggplot(tab_ins, aes('infractions', 'urgents')) +
     geom_point() + 
    theme(figure_size = (5, 4)))

Human rights matters haven't been an issue inspected in copper extraction companies. 

In [ ]:
tab_ins.humanrights.describe()

Just one company had urgent matters inspected.

In [ ]:
tab_ins[tab_ins['urgents']>45]

### Matters inspected

In [ ]:
idsfisc_copper = df_copper.idfiscalizacion
str1 = "','".join(idsfisc_copper)

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, crae, grupocodigotipomaterias2, grupocodigotipomaterias
    from raw.inspections_complete
    where idfiscalizacion in {};""".format("('" + str1 + "')")
df_matters = pd.read_sql_query(qry, engine)

In [ ]:
print (df_matters.shape)

In [ ]:
df_matters_melt = pd.melt(df_matters, id_vars = ['rutempresamask', 'crae'])
df_matters_melt = df_matters_melt[df_matters_melt['value'] != '99']
df_matters_melt.head()

In [ ]:
df_matters_melt.value.str.split(";").head()

### Matters of infractions

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select *
    from cleaned.violations
    limit 3;"""
pd.read_sql_query(qry, engine)

## Data story

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, sum(cast(urgencia as int)) as urgents, 
    sum(derechofund) as humanrights,
    sum(infra) as infractions, count(*) as inspections
    from raw.inspections_complete
    where ccae = '133000'
    group by rutempresamask;"""
tab_ins = pd.read_sql_query(qry, engine)

In [ ]:
id_company = ''.join(tab_ins[tab_ins['urgents']>45].rutempresamask)
print (id_company)

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select *
    from raw.taxes
    where rutmask = {};""".format("'" + id_company + "'")
tab_tax = pd.read_sql_query(qry, engine)

In [ ]:
tab_tax.ntrabajadores.mean()

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select *
    from raw.inspections_complete
    where rutempresamask = {};""".format("'" + id_company + "'")
tab_emp = pd.read_sql_query(qry, engine)
tab_emp.shape

In [ ]:
print('Total inspections:', tab_emp.idfiscalizacion.count())
print('Total proactive inspections:', sum(tab_emp["solicitante"] == 'Por Programa'))
print('Total reactive inspections:', sum(tab_emp["solicitante"] != 'Por Programa'))
print('Total urgent inspections:', tab_emp.urgencia.sum())
print('Total matters inspected:', tab_emp.num_materias.sum().astype('int'))
print('Total infractions:', tab_emp.infra.sum().astype('int'))
print('Total inspections with violations:', tab_emp.infractor.sum())

In [ ]:
tab_emp["infractor"].sum()
tab_emp.infractor.sum()

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask,  
    date(agno || '-' || mesreg || '-01') as date,    
    count(rutempresamask) as inspections, 
    sum(infra)  as infractions,
    sum(num_materias) as matters,
    sum(case when solicitante = 'Por Programa' then 1 else 0 end) as proactive,
    sum(case when solicitante = 'Por Programa' then 0 else 1 end) as reactive
    from raw.inspections_complete
    where rutempresamask = {} 
    group by rutempresamask, date
    order by date;""".format("'" + id_company + "'")
tab_summ = pd.read_sql_query(qry, engine)
tab_summ['date'] = pd.to_datetime(tab_summ.date)
print(tab_summ.shape)

In [ ]:
tab_summ['infractions'] = tab_summ['infractions'].astype('int')
tab_summ['matters'] = tab_summ['matters'].astype('int')
tab_summ['prop'] = round(100*tab_summ['infractions']/tab_summ['matters'])
tab_summ['prop'] = tab_summ['prop'].astype('int')
tab_summ['prop'] = tab_summ['prop'].map(str) + "%"
tab_summ.dtypes

In [ ]:
tab_summ.head()

In [ ]:
(ggplot(tab_summ, aes ( x = 'date', y = 'inspections')) + 
    geom_hline(yintercept = 0, color = 'gray') + 
    geom_bar(stat = 'identity', fill = 'purple', alpha = .5) + 
    geom_text(aes(y = 'inspections + 1', label = 'inspections'),
              color = 'black', size = 10) + 
    ylim(0, 20) + 
    ylab('Count') + 
    xlab('Month of inspection')  +
    #scale_x_datetime(breaks = date_breaks('1 month')) +
    scale_x_datetime(
         breaks=date_breaks('1 months'),
         labels=custom_date_format2) +
    theme(axis_text_x = element_text(angle = 90, size= 10), 
         figure_size = (25, 4)))

In [ ]:
tab_summ.head()
tab_gg = pd.melt(tab_summ[['rutempresamask', "date", "proactive","reactive", "inspections"]], 
        id_vars = ['rutempresamask', 'date', 'inspections'])
(ggplot(tab_gg, aes ( x = 'date', y = 'value', fill = 'variable')) + 
    geom_hline(yintercept = 0, color = 'gray') + 
    geom_bar(stat = 'identity', alpha = .5) + 
    geom_text(aes(y = 'inspections + 1', label = 'inspections'),
              color = 'black', size = 10) + 
    ylim(0, 20) + 
    ylab('Count') + 
    xlab('Month of inspection')  +
    scale_x_datetime(
         breaks=date_breaks('1 months'),
         labels=custom_date_format2) +
    theme(axis_text_x = element_text(angle = 90, size= 10), 
         figure_size = (25, 4)))

In [ ]:
(ggplot(tab_summ, aes ( x = 'date', y = -1)) + 
    geom_hline(yintercept = -1, color = 'gray') + 
    geom_linerange(aes(ymin =0, ymax = 'matters'), 
            color = 'blue') + 
    geom_linerange(aes(ymin =0, ymax = 'infractions'), 
            color = 'red') + 
    geom_point(aes(y = 'infractions'), 
            color = 'red', size = 4, 
               alpha = .3, shape  = 4) + 
    geom_text(aes(y = 'matters + 11', label = 'matters'), 
            size = 8, color = 'blue') + 
    geom_text(aes(y = 'matters + 6', label = 'prop'), 
            size = 6, color = 'gray') + 
    ylab('Count') + 
    xlab('Month of infractions') +
    #scale_x_datetime(breaks=date_breaks('1 months')) +
    scale_x_datetime(
         breaks=date_breaks('1 months'),
         labels=custom_date_format2) + 
    theme(axis_text_x = element_text(angle = 90, size= 10), 
         figure_size = (25, 4)))

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, crae, grupoglosatipomaterias2, grupoglosatipomaterias,
    date(agno || '-' || mesreg || '-01') as date   
    from raw.inspections_complete
    where rutempresamask = {};""".format("'" + id_company + "'")
df_matters = pd.read_sql_query(qry, engine)
df_matters['date'] = pd.to_datetime(df_matters.date)
df_matters.head()

In [ ]:
subdf_matt = df_matters[df_matters['date'] == '2014-11-01']
subdf_matt.head()
tt_melt = pd.melt(subdf_matt, id_vars = ['rutempresamask', 'crae', 'date'])
tt_melt = tt_melt[tt_melt['value'] != '99']
tt_melt.value.value_counts()

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, crae, grupoglosainfra, grupoglosainfra2,
    date(agno || '-' || mesreg || '-01') as date   
    from raw.inspections_complete
    where rutempresamask = {};""".format("'" + id_company + "'")
df_infra = pd.read_sql_query(qry, engine)
df_infra['date'] = pd.to_datetime(df_infra.date)
df_infra.head()

In [ ]:
subdf_infra = df_infra[df_infra['date'] == '2014-11-01']
subdf_infra.head()
tt_melt = pd.melt(subdf_infra, id_vars = ['rutempresamask', 'crae', 'date'])
tt_melt = tt_melt[tt_melt['value'] != '99']
tt_melt.value.value_counts()

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, crae, grupoglosainfra_det, grupoglosainfra2_det,
    date(agno || '-' || mesreg || '-01') as date   
    from raw.inspections_complete
    where rutempresamask = {};""".format("'" + id_company + "'")
df_infradet = pd.read_sql_query(qry, engine)
df_infradet['date'] = pd.to_datetime(df_infradet.date)
df_infradet.head()

In [ ]:
subdf_infrad = df_infradet[df_infradet['date'] == '2014-11-01']
subdf_infrad.head()
tt_melt = pd.melt(subdf_infrad, id_vars = ['rutempresamask', 'crae', 'date'])
tt_melt = tt_melt[tt_melt['value'] != '99']
tt_melt.value.value_counts()

In [ ]:
tt_melt = pd.melt(df_infradet, id_vars = ['rutempresamask', 'crae', 'date'])
tt_melt = tt_melt[tt_melt['value'] != '99']
tt_melt.value.value_counts()

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, crae, grupoCodigoNormaInfra2_det,
    date(agno || '-' || mesreg || '-01') as date   
    from raw.inspections_complete
    where rutempresamask = {};""".format("'" + id_company + "'")
df_infranorma = pd.read_sql_query(qry, engine)
df_infranorma['date'] = pd.to_datetime(df_infranorma.date)
df_infranorma.head()

In [ ]:
subdf_infranorma = df_infranorma[df_infranorma['date'] == '2014-11-01']
subdf_infranorma.head()
tt_melt = pd.melt(subdf_infranorma, id_vars = ['rutempresamask', 'crae', 'date'])
tt_melt = tt_melt[tt_melt['value'] != '99']
tt_melt.value.value_counts()

In [ ]:
tt_melt.value.str.split('|')

- GRAVISIMA	Do not keep the main company sanitary and environmental conditions necessary to protect the life and health of workers who work in your work, business or work

1305-h = 4

- GRAVISIMA	Do not implement the main company a management system of health and safety at work, in the work, work or services of their own business.

1305-a = 2

- GRAVISIMA	Do not monitor the main company compliance that falls to the contractors of the obligation to inform the tts. of the risks; control and prevention measures; methods of correct work and functioning of the Joint Committees and Dept. of Risk Prevention.

1307-b = 2

- GRAVISIMA	Do not make the main company a Labor Safety and Health Work Program.

1307-d = 1

- GRAVE	Do not count the extractive mining company with a safe work procedure for the operation, maintenance and repair of machinery.

1141-f = 1

In [ ]:
tt_melt.value.str.split('|').head()

#### Matters inspected

In [ ]:
qry = """ set role direccion_trabajo_inspections_write;
    select *
    from cleaned.inspected_matters_updatedbook
    limit 3;""".format("'" + id_company + "'")
plsql.query(qry, engine)

In [ ]:
qry = """ set role direccion_trabajo_inspections_write;
    select matter_code, count(*) as count
    from cleaned.inspected_matters_updatedbook
    where rutempresamask = {}
    group by matter_code
    order by count desc;""".format("'" + id_company + "'")
plsql.query(qry, engine)

In [ ]:
tab_matt.head()

In [ ]:
qry = """set role direccion_trabajo_inspections_write;
    select rutempresamask, crae, grupoCodigoNormaInfra2_det,
    date(agno || '-' || mesreg || '-01') as date   
    from raw.inspections_complete
    where rutempresamask = {};""".format("'" + id_company + "'")
df_infranorma = pd.read_sql_query(qry, engine)